In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import xlrd
import config

In [2]:
# Database connection parameters - replace with your actual parameters
dbname = config.dbname
user = config.user
password = config.password
host = config.host
port = config.port

In [3]:
# Load your data from the .xls file (replace with the correct path)
xls_file_path = 'data/static_data/type_ids.xlsx'
df = pd.read_excel(xls_file_path)
df = df.fillna(0)


In [4]:
print(df.head())

   TYPEID  GROUPID       TYPENAME  MARKETGROUPID
0       0        0        #System            0.0
1       2        2    Corporation            0.0
2       3        3         Region            0.0
3       4        4  Constellation            0.0
4       5        5   Solar System            0.0


In [7]:
df = df[df['MARKETGROUPID'] != 0]

In [8]:
print(df.head())

    TYPEID  GROUPID     TYPENAME  MARKETGROUPID
17      18      458  Plagioclase          516.0
18      19      461    Spodumain          517.0
19      20      457      Kernite          523.0
20      21      454   Hedbergite          527.0
21      22      450      Arkonor          512.0


In [9]:
def create_table(dbname, user, password, host, port, table_name):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()
    print("Check 2")
    # Create table if it doesn't exist
    cur.execute(sql.SQL("""
        CREATE TABLE IF NOT EXISTS {}
        (
            type_id INTEGER PRIMARY KEY,
            group_id INTEGER NOT NULL,
            type_name VARCHAR(100) NOT NULL,
            market_group_id INTEGER NOT NULL
        );
    """).format(sql.Identifier(table_name)))
    # Commit changes and close connection
    conn.commit()
    cur.close()
    conn.close()
    

In [10]:
create_table(dbname, user, password, host, port, 'commodities')

Check 2


In [11]:
def add_timepoint(dbname, user, password, host, port, data, table_name):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()
    # Insert data
    print(data.head())
    for _, row in data.iterrows():
        cur.execute(sql.SQL("""
            INSERT INTO {} (type_id, group_id, type_name, market_group_id)
            VALUES (%s, %s, %s, %s);
        """).format(sql.Identifier(table_name)), 
            (row['TYPEID'], row['GROUPID'], row['TYPENAME'], row['MARKETGROUPID']))
    # Commit changes and close connection
    conn.commit()
    cur.close()
    conn.close()

In [12]:
add_timepoint(dbname, user, password, host, port, df, 'commodities')


    TYPEID  GROUPID     TYPENAME  MARKETGROUPID
17      18      458  Plagioclase          516.0
18      19      461    Spodumain          517.0
19      20      457      Kernite          523.0
20      21      454   Hedbergite          527.0
21      22      450      Arkonor          512.0
